In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import scipy as sp
import scipy.stats
import pandas as pd
import numpy as np
from tqdm import tqdm
import itertools
import re
res_digit = r'[0-9]'

# fourier transform
from scipy.fft import fft, ifft

from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.manifold import TSNE



In [2]:
# This is a hack to make the library in the parent folder available for imoprts
# A better solution is by np8 here:
# https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
import os
import inspect

thisdir = sys.path[0]
print(f"thisdir = {thisdir}")
parentdir = os.path.dirname(thisdir)
#print(f"parentdir = {parentdir}")
if not parentdir in sys.path:
    print("Adding parent directory to python path")
    sys.path.insert(1, parentdir)
else:
    print("Skipping adding parent direct to path (there already)")

print(f"sys.path =\n{sys.path}")



thisdir = /home/luke/git/external/predicament/notebooks
Adding parent directory to python path
sys.path =
['/home/luke/git/external/predicament/notebooks', '/home/luke/git/external/predicament', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/luke/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3.10/dist-packages']


In [3]:
## ensure relative path to data directory is sound
# for the notebook we need to modify the BASE_DATA_FOLDER
import os 
os.environ['PREDICAMENT_DATA_DIR'] =  '../data'

from predicament.utils.config import DREEM_EEG_CHANNELS
from predicament.utils.config import RESULTS_BASE_PATH

In [4]:
from predicament.data.timeseries import create_participant_data_edf_only
from predicament.data.windowed import window_all_participants_data
from predicament.data.windowed import merge_condition_data
from predicament.data.partitioning import between_subject_cv_partition

from predicament.data.features import MAXIMAL_FEATURE_GROUP
from predicament.data.features import STATS_FEATURE_GROUP
from predicament.data.features import INFO_FEATURE_GROUP
from predicament.data.features import FREQ_FEATURE_GROUP
from predicament.data.features import convert_timeseries_to_features
from prepare_evaluation_data import load_dataframe_and_config


## Balanced data, held out participants

In [14]:
#res_fname = 'balanced_eeg_2023-12-11T10:13:02.csv'
#res_fname = 'balanced_eeg_2023-12-12T08:43:49.csv'
#res_fname = 'bayesian_optimization_random_forest_2023-12-20T18:40:43.csv'
#res_fname = 'bayesian_optimization_random_forest_2023-12-20T18:40:48.csv'
#res_fname = 'bayesian_optimization_RandomForest_2023-12-28T19:06:57.csv'
#res_fname = 'bayesian_optimization_RandomForest_2024-02-20T19:56:15.csv'
#res_fname = 'bayesian_optimization_RandomForest_2024-02-22T16:04:38.csv'
#res_fname = 'bayesian_optimization_RandomForest_2024-02-22T16:06:29.csv'
#res_fname = 'bayesian_optimization_SVC_2024-03-26T18:31:45.csv'
#res_fname = 'bayesian_optimization_SVC_2024-03-27T01:13:44.csv'
#res_fname = 'bayesian_optimization_SVC_2024-03-27T11:01:12.csv'
res_fname = 'bayesian_optimization_SVC_2024-03-27T13:42:44.csv'

import os
res_fpath = os.path.join(RESULTS_BASE_PATH, res_fname)
df = pd.read_csv(res_fpath)
df

,Unnamed: 0,model,held out,balanced,n_splits,feature set,mean_fit_time,std_fit_time,mean_score_time,std_score_time,...,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,split10_test_score,split11_test_score,mean_test_score,std_test_score,rank_test_score
0,0,SVC(),participant,True,12,"['Correlation', 'MAD', 'FreqSkewness', 'Lempel...",7.785361,0.322625,0.567776,0.140841,...,0.172117,0.171076,0.206564,0.217759,0.213531,0.271762,0.041353,0.191081,0.055287,29
1,1,SVC(),participant,True,12,"['Correlation', 'MAD', 'FreqSkewness', 'Lempel...",9.250951,0.314513,0.498111,0.101674,...,0.148021,0.052910,0.075290,0.000000,0.006342,0.000000,0.000000,0.091987,0.073429,50
2,2,SVC(),participant,True,12,"['Correlation', 'MAD', 'FreqSkewness', 'Lempel...",7.604487,0.496380,0.648158,0.264177,...,0.148021,0.052910,0.075290,0.000000,0.000000,0.000000,0.000000,0.092489,0.075116,38
3,3,SVC(),participant,True,12,"['Correlation', 'MAD', 'FreqSkewness', 'Lempel...",6.338025,0.166341,0.356534,0.094712,...,0.148021,0.052910,0.075290,0.000000,0.000000,0.000000,0.000000,0.092489,0.075116,38
4,4,SVC(),participant,True,12,"['Correlation', 'MAD', 'FreqSkewness', 'Lempel...",6.429352,0.162622,0.360852,0.092568,...,0.148021,0.052910,0.075290,0.000000,0.000000,0.000000,0.000000,0.092489,0.075116,38
5,5,SVC(),participant,True,12,"['Correlation', 'MAD', 'FreqSkewness', 'Lempel...",7.777599,0.254152,0.621281,0.166422,...,0.154905,0.102293,0.086873,0.000000,0.118393,0.023355,0.033835,0.115521,0.064623,33
6,6,SVC(),participant,True,12,"['Correlation', 'MAD', 'FreqSkewness', 'Lempel...",9.134998,0.222292,0.510200,0.125585,...,0.148021,0.052910,0.075290,0.000000,0.000000,0.000000,0.000000,0.092489,0.075116,38
7,7,SVC(),participant,True,12,"['Correlation', 'MAD', 'FreqSkewness', 'Lempel...",6.506679,0.155385,0.358240,0.096322,...,0.148021,0.052910,0.075290,0.000000,0.000000,0.000000,0.000000,0.092489,0.075116,38
8,8,SVC(),participant,True,12,"['Correlation', 'MAD', 'FreqSkewness', 'Lempel...",8.039242,0.232723,0.611601,0.163611,...,0.182444,0.178131,0.235521,0.238901,0.213531,0.246285,0.120301,0.210914,0.037928,15
9,9,SVC(),participant,True,12,"['Correlation', 'MAD', 'FreqSkewness', 'Lempel...",7.648128,0.208011,0.645616,0.166626,...,0.197935,0.195767,0.227799,0.128964,0.105708,0.210191,0.022556,0.182690,0.068881,30


In [15]:
# find the best performing row for each model type
# and extract test scores

cols_of_interest = [ col for col in df.columns if col.startswith('split') and col.endswith('test_score')]

model_to_scores = {}
for model in np.unique(df['model']):
    print(f"model = {model}")
    max_model_score = df[df['model'] == model]['mean_test_score'].max()
    res = df[(df['model'] == model) & (df['mean_test_score'] == max_model_score)]
    res = res[cols_of_interest]
    test_scores = res.to_numpy()
    model_to_scores[model] = test_scores
#     res = dict(res.iloc[0])


model = SVC()


Null hypothesis is that the mean score is at chance level. Namely, the probability of guessing the correct class at random or worse.
$$H_0: \mu \leq \mu_0 = 0.2$$

The alternative hypothesis is that it is better than chance:
$$H_1: \mu > \mu_0$$

I don't have a variance for my data, so I am forced to use the hypothesis test for unknown variance. As I am predicting one way, it is a one tailed test. The test statistic is therefore:
$$TS = \sqrt{n} \frac{\bar{X}- \mu_0}{S}$$
where $S$ is the sample standard deviation.


In [16]:
alpha = 0.05
import scipy.stats
mu0 = 0.2
for model, scores in model_to_scores.items():
    print(f"model: {model}")
    Xbar = np.mean(scores)
    S = np.std(scores, ddof=1)
    print(f"Xbar = {Xbar}, S = {S}")
    TS = np.sqrt(12)*(Xbar - mu0)/S
    print(f"TS = {TS}")
    t_11 =  scipy.stats.t.ppf(1-alpha, scores.size-1, loc=mu0, scale=S)
    print(f"Threshold mean score to reject H_0 is {t_11:.4f}, mean score is {TS:.4f}")
    if t_11 < TS:
        print(f"At {100*(1-alpha)}% significance level we can reject H_0")
    pval = 1 - scipy.stats.t.cdf(TS, scores.size-1, loc=mu0, scale=S )
    print(f"We can reject H0 at any significance level greater than {pval:.3g}")
    print(f"We can reject H0 at any confidence level less than {100*(1-pval):.1f}")
    print()

model: SVC()
Xbar = 0.2307622767661798, S = 0.08571562817995944
TS = 1.2432231425441846
Threshold mean score to reject H_0 is 0.3539, mean score is 1.2432
At 95.0% significance level we can reject H_0
We can reject H0 at any significance level greater than 5.03e-08
We can reject H0 at any confidence level less than 100.0



In [17]:
list(range(0,100,10))

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90]

## Tests to develop

| Test | Motivation |
| :---: |  :---: | 
| Hold one subject out balanced | Can test that there is some information for prediction between partitipants |
| Hold one subject-condition out (balanced) | Can test that there is some information for prediction within partitipants with unseen conditions |
| Hold one subject-condition-phase out (balanced) | Can test that there is some information for prediction within partitipant-condition |
